# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
!pip install kagglehub
!pip uninstall psutil -y
!pip install psutil==5.9.0
!pip uninstall -y ipywidgets azureml-widgets
!pip install ipywidgets==7.6.5 azureml-widgets --no-cache-dir

Found existing installation: psutil 5.9.0
Uninstalling psutil-5.9.0:
  Successfully uninstalled psutil-5.9.0
  Using cached psutil-5.9.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (281 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-nvdashboard 0.13.0 requires jupyterlab>=4, but you have jupyterlab 3.6.8 which is incompatible.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
azureml-training-tabular 1.60.0 requires scipy<1.11.0,>=1.0.0, but you have scipy 1.11.0 which is incompatible.
azureml-automl-dnn-nlp 1.60.0 requires torch==2.2.2, but you have torch 2.6.0 which is incompatible.
Found existing installation: ipywidgets 7.6.5


In [2]:
import os
import shutil
import logging

import kagglehub
import pandas as pd

from azureml.core import Workspace, Experiment, Dataset
from azureml.core.authentication import AzureCliAuthentication
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
# Fetch data from Kaggle and move it to src folder of project
source_path = kagglehub.dataset_download("blastchar/telco-customer-churn")
print("Download Path to dataset files:", source_path)

shutil.move(source_path, os.path.join(os.getcwd(),'data'))
print("New Path to dataset files:", os.path.join(os.getcwd(),'data'))

data_file = 'WA_Fn-UseC_-Telco-Customer-Churn.csv'

ws = Workspace.from_config()


default_ds = ws.get_default_datastore()
default_ds.upload_files(files=[os.path.join(os.getcwd(),'data',data_file)],
                        target_path='data', # Directory on the datastore
                        overwrite=True, # Overwrite if a file with the same name exists
                        show_progress=True)

unregistered_tabular_data = Dataset.Tabular.from_delimited_files(
    path=[(default_ds, os.path.join('data',data_file))],
    validate=True, # Validates data schema during creation
    separator=','  # Specify your CSV separator
)                        

Download Path to dataset files: /home/azureuser/.cache/kagglehub/datasets/blastchar/telco-customer-churn/versions/1
New Path to dataset files: /mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook283859/code/Users/odl_user_283859/data
Uploading an estimated of 1 files
Uploading /mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook283859/code/Users/odl_user_283859/data/WA_Fn-UseC_-Telco-Customer-Churn.csv
Uploaded /mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook283859/code/Users/odl_user_283859/data/WA_Fn-UseC_-Telco-Customer-Churn.csv, 1 files out of an estimated total of 1
Uploaded 1 files


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [4]:
unregistered_tabular_data = unregistered_tabular_data.drop_columns(columns=['customerID'])

try:
    registered_dataset = unregistered_tabular_data.register(
        workspace=ws,
        name='ibm-telco-data',
        description="Cleaned data for churn prediction AutoML",
        create_new_version=True
    )
    print(f"Dataset '{registered_dataset.name}' registered successfully.")
    print(f"Version: {registered_dataset.version}")
    print(f"Registered Dataset ID: {registered_dataset.id}")

except Exception as e:
    print(f"Error registering dataset: {e}")
    print("Attempting to get existing dataset instead...")
    try:
        registered_dataset = Dataset.get_by_name(ws, name='ibm-telco-data', version='1')
        print(f"Using existing dataset '{registered_dataset.name}', version {registered_dataset.version}")
    except Exception as get_e:
        print(f"Failed to get existing dataset after registration error: {get_e}")

Dataset 'ibm-telco-data' registered successfully.
Version: 1
Registered Dataset ID: a8153869-ea1a-4ee0-a8cd-20adef6db37e


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
amlcompute_cluster_name = "mycompute"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6,
                                                           min_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

# choose a name for experiment
experiment_name = 'churn-auto-ml'

experiment=Experiment(ws, experiment_name)

# # # TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}
# # TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target="mycompute",
    task="classification",
    training_data=registered_dataset,
    label_column_name='Churn',
    iterations=30,
    iteration_timeout_minutes=5,
    enable_early_stopping= True,
    featurization= 'auto',
    enable_onnx_compatible_models=True,
    verbosity=logging.INFO,
    debug_log = "automl_errors.log",
    **automl_settings)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded........................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on mycompute with default configuration
Running on remote compute: mycompute


Experiment,Id,Type,Status,Details Page,Docs Page
churn-auto-ml,AutoML_128ce24d-325c-41d4-8bc7-5cd49b359685,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Received interrupt. Returning now.

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

2025-06-21 19:11:08.282114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750533069.903666   31089 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750533070.363458   31089 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750533074.727667   31089 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750533074.727723   31089 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750533074.727726   31089 computation_placer.cc:177] computation placer alr

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
churn-auto-ml,AutoML_6efda4b1-e969-42c2-89ce-f771f424ee6f,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of missing feature values that may be more appropriate based on the data type and business requirement.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization
DETAILS:      
+------------------------------+------------------------------+------------------------------+
|Column name          

{'runId': 'AutoML_6efda4b1-e969-42c2-89ce-f771f424ee6f',
 'target': 'mycompute',
 'status': 'Completed',
 'startTimeUtc': '2025-06-21T18:44:22.769418Z',
 'endTimeUtc': '2025-06-21T19:10:06.21752Z',
 'services': {},
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'mycompute',
  'AMLSettingsJsonString': '{"path":null,"name":"churn-auto-ml","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-283845","workspace_name":"quick-starts-ws-283845","region":"southcentralus","compute_target":"mycompute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_run, fitted_model = remote_run.get_output()

print(f"Best AutoML run ID: {best_run.id}")
print(f"Best model pipeline type: {type(fitted_model)}")

# Get metrics from the best run
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n Parameters:',parameter_values)

Best AutoML run ID: AutoML_6efda4b1-e969-42c2-89ce-f771f424ee6f_29
Best model pipeline type: <class 'azureml.training.tabular.models.pipeline_with_ytransformations.PipelineWithYTransformations'>
Best Run Id:  AutoML_6efda4b1-e969-42c2-89ce-f771f424ee6f_29

 Accuracy: 0.8024993547970837

 Parameters: []


In [12]:
#TODO: Save the best model
model_file_name = 'automlBestModel.pkl'
remote_model_path = f'outputs/model.pkl'
local_model_path = os.path.join('./training', model_file_name)

print(f"Downloading model from run {best_run.id} to {local_model_path}...")
best_run.download_file(name=remote_model_path, output_file_path=local_model_path)
print("Model downloaded successfully.")

print(f"Registering model '{model_file_name}' from run {best_run.id}...")
model = Model.register(workspace=ws,
                       model_path=local_model_path,
                       model_name="best_automl_model",
                       tags={'run_id': best_run.id, 'accuracy': best_run.get_metrics().get('accuracy', 'N/A')},
                       description="Best model tuned with automl")

print(f"Model registered successfully with name: {model.name}, version: {model.version}")

Model downloaded successfully.
Registering model 'automlBestModel.pkl' from run AutoML_6efda4b1-e969-42c2-89ce-f771f424ee6f_29...
Registering model best_automl_model
Model registered successfully with name: best_automl_model, version: 2


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
